Daria Vasileva


In this problem I will ask you to use data from the following dataset taken from the UCI dataset repository: http://archive.ics.uci.edu/ml/datasets/Adult As the original task of the dataset lays out, the goal is to try and classify whether or not an individual is likely to make more or less than 50K per year. Fill in the jupyter notebook and post a link to a GitHub classroom solution. Please share it with me.

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
%matplotlib inline

In [25]:
income_df=pd.read_csv('adult.csv')

income_df.head()


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [26]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [28]:
income_df.isin(['?']).sum(axis=0)

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [30]:
#Replacing the special character to nan and then drop the columns
income_df['native-country'] = income_df['native-country'].replace('?',np.nan)
income_df['workclass'] = income_df['workclass'].replace('?',np.nan)
income_df['occupation'] = income_df['occupation'].replace('?',np.nan)
#Dropping the NaN rows now 
income_df.dropna(how='any',inplace=True)

In [31]:
income_df.isin(['?']).sum(axis=0)

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [32]:
#standard quantities in dataset
income_df.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,1.897347e+05,10.118460,1101.430344,88.595418,40.938017
std,13.217870,1.056392e+05,2.552881,7506.430084,404.956092,12.007508
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.173882e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783160e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.379260e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [33]:
#converting the given dataset(age) into float type
income_df.age = income_df.age.astype(float)
income_df['hours-per-week'] = income_df['hours-per-week'].astype(float)

In [34]:
#storing the new dataset in my_df(data without dropped columns)
my_df = income_df.dropna()

In [35]:
my_df['predclass'] = my_df['income']
del my_df['income']
my_df['education-num'] = my_df['educational-num']
del my_df['educational-num']

In [36]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             45222 non-null  float64
 1   workclass       45222 non-null  object 
 2   fnlwgt          45222 non-null  int64  
 3   education       45222 non-null  object 
 4   marital-status  45222 non-null  object 
 5   occupation      45222 non-null  object 
 6   relationship    45222 non-null  object 
 7   race            45222 non-null  object 
 8   gender          45222 non-null  object 
 9   capital-gain    45222 non-null  int64  
 10  capital-loss    45222 non-null  int64  
 11  hours-per-week  45222 non-null  float64
 12  native-country  45222 non-null  object 
 13  predclass       45222 non-null  object 
 14  education-num   45222 non-null  int64  
dtypes: float64(2), int64(4), object(9)
memory usage: 5.5+ MB


In [37]:
#reading each column entry
print('workclass',my_df.workclass.unique())
print('education',my_df.education.unique())
print('marital-status',my_df['marital-status'].unique())
print('occupation',my_df.occupation.unique())
print('relationship',my_df.relationship.unique())
print('race',my_df.race.unique())
print('gender',my_df.gender.unique())
print('native-country',my_df['native-country'].unique())
print('predclass',my_df.predclass.unique())

workclass ['Private' 'Local-gov' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay']
education ['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' '5th-6th' 'Assoc-voc' '9th' 'Doctorate'
 '12th' '1st-4th' 'Preschool']
marital-status ['Never-married' 'Married-civ-spouse' 'Widowed' 'Separated' 'Divorced'
 'Married-spouse-absent' 'Married-AF-spouse']
occupation ['Machine-op-inspct' 'Farming-fishing' 'Protective-serv' 'Other-service'
 'Prof-specialty' 'Craft-repair' 'Adm-clerical' 'Exec-managerial'
 'Tech-support' 'Sales' 'Priv-house-serv' 'Transport-moving'
 'Handlers-cleaners' 'Armed-Forces']
relationship ['Own-child' 'Husband' 'Not-in-family' 'Unmarried' 'Wife' 'Other-relative']
race ['Black' 'White' 'Other' 'Amer-Indian-Eskimo' 'Asian-Pac-Islander']
gender ['Male' 'Female']
native-country ['United-States' 'Peru' 'Guatemala' 'Mexico' 'Dominican-Republic'
 'Ireland' 'Germany' 'Philippines' 'Thailand' 'Haiti' 'El-Salv

In [38]:
#creating two classes '<=50k' and '>50k'
my_df.loc[income_df['income'] == ' >50K', 'predclass'] = 1
my_df.loc[income_df['income'] == ' <=50K', 'predclass'] = 0

In [39]:
#properly classifying the data with education column entries
my_df['education'].replace('Preschool', 'dropout',inplace=True)
my_df['education'].replace('10th', 'dropout',inplace=True)
my_df['education'].replace('11th', 'dropout',inplace=True)
my_df['education'].replace('12th', 'dropout',inplace=True)
my_df['education'].replace('1st-4th', 'dropout',inplace=True)
my_df['education'].replace('5th-6th', 'dropout',inplace=True)
my_df['education'].replace('7th-8th', 'dropout',inplace=True)
my_df['education'].replace('9th', 'dropout',inplace=True)
my_df['education'].replace('HS-Grad', 'HighGrad',inplace=True)
my_df['education'].replace('HS-grad', 'HighGrad',inplace=True)
my_df['education'].replace('Some-college', 'CommunityCollege',inplace=True)
my_df['education'].replace('Assoc-acdm', 'CommunityCollege',inplace=True)
my_df['education'].replace('Assoc-voc', 'CommunityCollege',inplace=True)
my_df['education'].replace('Bachelors', 'Bachelors',inplace=True)
my_df['education'].replace('Masters', 'Masters',inplace=True)
my_df['education'].replace('Prof-school', 'Masters',inplace=True)
my_df['education'].replace('Doctorate', 'Doctorate',inplace=True)

In [40]:
#similarly classifying martial status column entries
my_df['marital-status'].replace('Never-married', 'NotMarried',inplace=True)
my_df['marital-status'].replace(['Married-AF-spouse'], 'Married',inplace=True)
my_df['marital-status'].replace(['Married-civ-spouse'], 'Married',inplace=True)
my_df['marital-status'].replace(['Married-spouse-absent'], 'NotMarried',inplace=True)
my_df['marital-status'].replace(['Separated'], 'Separated',inplace=True)
my_df['marital-status'].replace(['Divorced'], 'Separated',inplace=True)
my_df['marital-status'].replace(['Widowed'], 'Widowed',inplace=True)

In [41]:
#Assigning the numeric values to the string type variables
number = LabelEncoder()
my_df['workclass'] = number.fit_transform(my_df['workclass'])
my_df['education'] = number.fit_transform(my_df['education'])
my_df['marital-status'] = number.fit_transform(my_df['marital-status'])
my_df['occupation'] = number.fit_transform(my_df['occupation'])
my_df['relationship'] = number.fit_transform(my_df['relationship'])
my_df['race'] = number.fit_transform(my_df['race'])
my_df['gender'] = number.fit_transform(my_df['gender'])
my_df['native-country'] = number.fit_transform(my_df['native-country'])
my_df['predclass'] = number.fit_transform(my_df['predclass'])

In [42]:
my_df['age_bin'] = pd.cut(my_df['age'], 20)

In [43]:
my_df['hours-per-week_bin'] = pd.cut(my_df['hours-per-week'], 10)
my_df['hours-per-week'] = my_df['hours-per-week']

In [44]:
my_df[['predclass', 'age']].groupby(['predclass'], as_index=False).mean().sort_values(by='age', ascending=False)

,predclass,age
1,1,44.006067
0,0,36.749427


In [45]:
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split #training and testing data split

In [46]:
my_df = my_df.apply(LabelEncoder().fit_transform)
my_df.head()

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,predclass,education-num,age_bin,hours-per-week_bin
0,8,2,18119,5,1,6,3,2,1,0,0,39,38,0,6,2,3
1,21,2,3913,3,0,4,0,4,1,0,0,49,38,0,8,5,4
2,11,1,23770,1,0,10,0,4,1,0,0,39,38,1,11,3,3
3,27,2,10474,1,0,6,0,2,1,96,0,39,38,1,9,7,3
5,17,2,15397,5,1,7,1,4,1,0,0,29,38,0,5,4,2


In [47]:
my_df['age-hours'] = my_df['age']*my_df['hours-per-week']
my_df['age-hours_bin'] = pd.cut(my_df['age-hours'], 10)

In [48]:
drop_elements = ['education', 'native-country', 'predclass', 'age_bin', 'age-hours_bin','hours-per-week_bin']
y = my_df["predclass"]
X = my_df.drop(drop_elements, axis=1)
X.head()

,age,workclass,fnlwgt,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,education-num,age-hours
0,8,2,18119,1,6,3,2,1,0,0,39,6,312
1,21,2,3913,0,4,0,4,1,0,0,49,8,1029
2,11,1,23770,0,10,0,4,1,0,0,39,11,429
3,27,2,10474,0,6,0,2,1,96,0,39,9,1053
5,17,2,15397,1,7,1,4,1,0,0,29,5,493


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,train_size=0.6, random_state=0)

In [50]:
#using standard scaler we can normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [51]:
from sklearn import linear_model

In [52]:
# Decision Tree
DT = DecisionTreeClassifier()
DT.fit(X_train,y_train)
y_pred = DT.predict(X_test)
score_DT = DT.score(X_test,y_test)
print("The accuracy of the Decision tree model is ",score_DT)
targets = ['<=50k' , '>50k']
print(classification_report(y_test, y_pred,target_names=targets))

The accuracy of the Decision tree model is  0.8018132566753275
              precision    recall  f1-score   support

       <=50k       0.87      0.86      0.87     13541
        >50k       0.60      0.62      0.61      4548

    accuracy                           0.80     18089
   macro avg       0.74      0.74      0.74     18089
weighted avg       0.80      0.80      0.80     18089



In [53]:
# K-Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
score_knn = knn.score(X_test,y_test)
print('The accuracy of the KNN Model is',score_knn)
targets = ['<=50k' , '>50k']
print(classification_report(y_test, y_pred,target_names=targets))

The accuracy of the KNN Model is 0.8307811377079993
              precision    recall  f1-score   support

       <=50k       0.87      0.90      0.89     13541
        >50k       0.68      0.61      0.65      4548

    accuracy                           0.83     18089
   macro avg       0.78      0.76      0.77     18089
weighted avg       0.83      0.83      0.83     18089



In [54]:
# Support Vector Classifier (SVM/SVC)
from sklearn.svm import SVC
svc = SVC(gamma=0.22)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
score_svc = svc.score(X_test,y_test)
print('The accuracy of SVC model is', score_svc)
targets = ['<=50k' , '>50k']
print(classification_report(y_test, y_pred,target_names=targets))

The accuracy of SVC model is 0.8454309248714689
              precision    recall  f1-score   support

       <=50k       0.87      0.94      0.90     13541
        >50k       0.76      0.57      0.65      4548

    accuracy                           0.85     18089
   macro avg       0.81      0.75      0.77     18089
weighted avg       0.84      0.85      0.84     18089



In [55]:
# Logistic Regression
LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
score_LR = LR.score(X_test,y_test)
print('The accuracy of the Logistic Regression model is', score_LR)
targets = ['<=50k' , '>50k']
print(classification_report(y_test, y_pred,target_names=targets))

The accuracy of the Logistic Regression model is 0.8271325114710597
              precision    recall  f1-score   support

       <=50k       0.85      0.93      0.89     13541
        >50k       0.71      0.53      0.61      4548

    accuracy                           0.83     18089
   macro avg       0.78      0.73      0.75     18089
weighted avg       0.82      0.83      0.82     18089



In [57]:
tabular_form = {'CLASSIFICATION':['LogisticRegression','SupportVectorClassifier','DecisionTree','K-NearestNeighbors'],
                'ACCURACY':[score_LR,score_svc,score_DT,score_knn]
                }
Tabular_form = pd.DataFrame(tabular_form,columns= ['CLASSIFICATION','ACCURACY'])
print(Tabular_form)

            CLASSIFICATION  ACCURACY
0       LogisticRegression  0.827133
1  SupportVectorClassifier  0.845431
2             DecisionTree  0.801813
3       K-NearestNeighbors  0.830781
